In [7]:
# dependencies
!pip install sentence-transformers faiss-cpu openai nltk

# imports
import os, re
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import openai

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)



True

In [ ]:
# for API key
os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API Key: ")

In [ ]:
# preprocess
def preprocess_text(text):
    return re.sub(r'\s+', ' ', text.replace('\n', ' ').strip())

def chunk_text_by_sentences(text, max_chars=800):
    sentences = sent_tokenize(text)
    chunks, current, length = [], [], 0
    for sent in sentences:
        sent = preprocess_text(sent)
        if length + len(sent) <= max_chars:
            current.append(sent)
            length += len(sent)
        else:
            chunks.append(' '.join(current)); current, length = [sent], len(sent)
    if current: chunks.append(' '.join(current))
    return chunks

In [ ]:
#  RAG Class
class ModernRAG:
    def __init__(self, model_name="all-MiniLM-L6-v2", embedding_dim=384):
        self.model = SentenceTransformer(model_name)
        self.embedding_dim = embedding_dim
        self.index = None
        self.metadata = []

    def build_index(self, chunks, sources):
        embeddings = self.model.encode(chunks, convert_to_numpy=True).astype('float32')
        faiss.normalize_L2(embeddings)
        self.index = faiss.IndexFlatIP(self.embedding_dim)
        self.index.add(embeddings)
        for c, s in zip(chunks, sources):
            self.metadata.append({'text': c, 'source': s})

    def query(self, question, top_k=3):
        q_emb = self.model.encode([preprocess_text(question)], convert_to_numpy=True).astype('float32')
        faiss.normalize_L2(q_emb)
        D, I = self.index.search(q_emb, top_k)
        return [(float(score), self.metadata[idx]) for score, idx in zip(D[0], I[0])]

    def build_prompt(self, question, retrieved):
        context = "\n".join([f"- {m['text']}" for _, m in retrieved])
        return f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"

    def call_openai(self, prompt):
        client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        resp = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=256,
            temperature=0
        )
        return resp.choices[0].message.content.strip()

In [ ]:
#  Example Workflow
rag = ModernRAG()

#  sample documents
open('sample1.txt','w').write("Space exploration began in 1957 with Sputnik. Humans landed on the Moon in 1969.")
open('sample2.txt','w').write("Mars missions: Viking, Pathfinder, Spirit and Opportunity. Perseverance launched in 2020.")

# Ingest
chunks, sources = [], []
for fp in ["sample1.txt", "sample2.txt"]:
    text = open(fp).read()
    cs = chunk_text_by_sentences(text, 400)
    chunks.extend(cs)
    sources.extend([fp]*len(cs))
rag.build_index(chunks, sources)

In [ ]:
# Query
question = "When did humans land on the Moon?"
retrieved = rag.query(question)
print("Retrieved:")
for s, m in retrieved:
    print(f"{s:.4f} | {m['text']}")

# Prompt + LLM Answer
prompt = rag.build_prompt(question, retrieved)
print("\nPrompt:\n", prompt)
print("\nAnswer:\n", rag.call_openai(prompt))